In [154]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import myUtils as utils
tech = "warm"  #biogas,battery, biomass, ror, geothermal，transmission
rootPath = f"../../results/scenario/{tech}"
pathTail = "/networks/base_s_27_3H_3H_2045.nc"


plt.style.use('bmh')
%matplotlib inline

nBase = pypsa.Network(f"../../results/scenario/base{pathTail}")
nMean = pypsa.Network(f"{rootPath}_mean/{pathTail}")
nGood = pypsa.Network(f"{rootPath}_good/{pathTail}")
nBest = pypsa.Network(f"{rootPath}_best/{pathTail}")

networks = {
  "nBase": nBase,
  "nMean": nMean,
  "nGood": nGood,
  "nBest": nBest
}

INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [157]:
import pandas as pd
import myUtils as utils
import importlib
importlib.reload(utils)


def getDfValue (n):
  lists = [
    n.statistics.capex().sum() / 1e9,
    n.statistics.capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_grouper").sum() / 1e9,

    n.statistics.capex(groupby="de_all_generator_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_all_generator_grouper").sum() / 1e9,
  ]
  return lists

data = {
  'Name': [
    "CAPEX(inkl. Nachbarländer)",
    "CAPEX",
    "investierte CAPEX",
    "OPEX",

    "Strom CAPEX",
    "Strom Opex"
  ],
  # 'Unit': [
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR'
  # ],
  'Base': getDfValue(nBase),
  'Mean': getDfValue(nMean),
  'Optimist': getDfValue(nGood),
  'Best': getDfValue(nBest)
}
print(len(data['Name']), len(data['Name']), len(data['Base']))



df = pd.DataFrame(data).set_index('Name')

caepx_base = df.loc['CAPEX', 'Base']
caepx_changes = (df.loc['CAPEX', ['Mean', 'Optimist', 'Best']] - caepx_base) / caepx_base * 100

elc_caepx_base = df.loc['Strom CAPEX', 'Base']
elc_caepx_changes = (df.loc['Strom CAPEX', ['Mean', 'Optimist', 'Best']] - elc_caepx_base) / elc_caepx_base * 100

table = utils.dataframe_to_table(df)
print(table)
print(caepx_changes)
print(elc_caepx_changes)

6 6 6
  [CAPEX(inkl. Nachbarländer)], [637,328], [626,019], [639,154], [592,662],
  [CAPEX], [74,925], [74,412], [73,622], [68,137],
  [investierte CAPEX], [63,338], [62,825], [62,035], [56,551],
  [OPEX], [5,494], [5,458], [5,226], [5,170],
  [Strom CAPEX], [35,375], [35,030], [35,183], [34,420],
  [Strom Opex], [0,131], [0,091], [0,088], [0,096],
Mean       -0.68488
Optimist   -1.73886
Best       -9.05925
Name: CAPEX, dtype: float64
Mean       -0.97536
Optimist   -0.54223
Best       -2.70108
Name: Strom CAPEX, dtype: float64


In [156]:
import pandas as pd
import myUtils as utils
import importlib
importlib.reload(utils)

df = pd.DataFrame()

for key, n in networks.items():
  generator_df = utils.getNByGroup(n, 'Generator', utils.de_generator_grouper)
  generators = generator_df.groupby('carrier').sum().p_nom_opt / 1e3
  g_links_df = utils.getNByGroup(n, 'Link', utils.de_generator_grouper)
  generatorLinks = g_links_df.groupby('carrier').sum().p_nom_opt / 1e3

  h2_df = utils.getNByGroup(n, 'Link', utils.de_h2_dispatch_grouper)
  h2s =  h2_df.groupby('carrier').sum().p_nom_opt / 1e3

  biomass = utils.getNByGroup(n,'Link',utils.de_biomass_chp)
  # print(biomass)
  s = pd.concat([generators, generatorLinks,h2s])
  s['Offwind'] = s['offwind-ac'] + s['offwind-dc'] + s['offwind-float']
  s['Solar'] = s['solar'] + s['solar-hsat']
  s['Onwind'] = s['onwind']
  

  s['Solardach'] = s['solar rooftop']
  s['Erdgas CHP'] = s['urban central CHP']
  s['Geothermal'] = s.get('geothermal organic rankine cycle', 0)
  s['Biomasse CHP'] = biomass.p_nom_opt.sum() 
  s['Gesamt'] = s.sum()
  
  s = s.reindex(['Onwind', 'Offwind', 'Solar', 'Solardach','OCGT','Erdgas CHP', 'Geothermal','H2 Fuel Cell', 'H2 turbine', 'Biomasse CHP', 'ror', 'Gesamt'])
  df[key] = s

# df = df[(df != 0).any(axis=1)]
df = df.fillna(0)


print(utils.dataframe_to_table(df))

print(df)


  [Onwind], [83,101], [82,851], [84,843], [80,550],
  [Offwind], [78,329], [78,329], [78,329], [78,329],
  [Solar], [50,287], [50,287], [50,287], [50,287],
  [Solardach], [107,327], [108,896], [111,848], [113,793],
  [OCGT], [73,804], [88,012], [74,152], [67,436],
  [Erdgas CHP], [61,302], [38,509], [41,941], [33,686],
  [Geothermal], [0,000], [0,000], [0,000], [0,000],
  [H2 Fuel Cell], [0,000], [0,000], [0,000], [0,000],
  [H2 turbine], [0,000], [0,000], [0,000], [0,000],
  [Biomasse CHP], [0,000], [0,000], [0,000], [0,000],
  [ror], [0,000], [0,000], [0,000], [0,000],
  [Gesamt], [834,497], [805,757], [808,649], [780,726],
                 nBase     nMean     nGood     nBest
carrier                                             
Onwind        83.10055  82.85098  84.84339  80.55005
Offwind       78.32878  78.32878  78.32878  78.32878
Solar         50.28742  50.28734  50.28731  50.28728
Solardach    107.32740 108.89603 111.84786 113.79262
OCGT          73.80419  88.01222  74.15171  67.4